In [1]:
# Requests
import requests as rq

# Data analysis
import pandas as pd
import random as rand

## Variables with calls

In [2]:
serv = "http://178.63.49.197:8080/spine-ws"
auth = "/login/authenticate"
latestVersion = "/external/dataset/latestVersion"
search = "/external/concepts/search"
direct = "/external/concept-to-concept/direct"
direct_count = "/external/direct-connections-count"

## Load table with drug-disease associations

In [3]:
drug_disease_filename = "./data/drugs/drug_disease.tsv"
drug_disease = pd.read_csv(drug_disease_filename, sep = "\t", comment = "#")
drug_disease.head()

,Disease,Drug,Drugbank ID,Palliative,In FDA label,Relative Efficacy (RE),Indication (from DailyMed),Note
0,adenocarcinoma,Erlotinib,DB00530,0,1,"0,867925",TARCEVA is a kinase inhibitor indicated for: F...,"Nearly 40% of lung cancers are adenocarcinoma,..."
1,adenocarcinoma,Floxuridine,DB00322,1,1,NaN,"Floxuridine for Injection, USP is effective in...",For palliative management of gastrointestinal ...
2,adenocarcinoma,Gefitinib,DB00317,0,1,"0,752137",IRESSA is a tyrosine kinase inhibitor indicate...,"Nearly 40% of lung cancers are adenocarcinoma,..."
3,alzheimer disease,Donepezil,DB00843,0,1,"0,538462",Donepezil hydrochloride is an acetylcholineste...,NaN
4,alzheimer disease,Galantamine,DB00674,0,1,NaN,Galantamine hydrobromide tablet is a cholinest...,NaN


## Login

In [4]:
loginInfo = {"password": "6cPBng68m", "username": "r.a.kudrin@gmail.com"}
resp_auth = rq.post(serv + auth, json = loginInfo)
auth_dict = resp_auth.json()
token = auth_dict["token"]
token_dict = {"x-token": token}
# print token_dict

## Get the ids

In [191]:
diseases = drug_disease["Disease"]
drug_ids = drug_disease["Drugbank ID"]

disease = rand.choice(diseases)
substance = rand.choice(drug_ids)
print disease, "WHOOOP!", substance

query_disease = "term:'%s'" % disease
query_drug = "term:'%s'" % substance

json_disease = {"additionalFields": [], "queryString": query_disease, "searchType": "TOKEN"}
json_drug = {"additionalFields": [], "queryString": query_drug, "searchType": "TOKEN"}

resp_disease = requests.post(serv + search, json = json_disease, headers = token_dict)
resp_drug = requests.post(serv + search, json = json_drug, headers = token_dict)

epilepsy WHOOOP! DB01611


In [193]:
resp_disease.json()

{u'content': [{u'id': u'8218', u'name': u'epilepsy, opercular'},
  {u'id': u'9271', u'name': u'grand mal status epilepticus'},
  {u'id': u'9387', u'name': u'reflex epilepsy, reading'},
  {u'id': u'13811', u'name': u'vesicle-fusing atpase (rattus norvegicus)'},
  {u'id': u'25670', u'name': u'epilepsy telangiectasia'},
  {u'id': u'26519',
   u'name': u'spastic paraplegia, epilepsy, and mental retardation'},
  {u'id': u'45171', u'name': u'epilepsy, cingulate'},
  {u'id': u'49693', u'name': u'infantile spasms'},
  {u'id': u'51751', u'name': u'epilepsy, reflex'},
  {u'id': u'57936', u'name': u'epilepsy, partial, motor'},
  {u'id': u'59278', u'name': u'amygdalo-hippocampal epilepsy'},
  {u'id': u'85809', u'name': u'epilepsy, tonic'},
  {u'id': u'86054', u'name': u'myoclonic epilepsies, progressive'},
  {u'id': u'119716', u'name': u'benign neonatal epilepsy, nonfamilial'},
  {u'id': u'122398', u'name': u'writing-induced reflex epilepsy'},
  {u'id': u'129835', u'name': u'status epilepticus'},


## Problem 1

In [156]:
json_direct = {
  "additionalFields": [
        "concept0Name",
        "directionalTriples"
  ],
  "leftInputs": [
    "2060394"
  ],
  "relationshipWeightAlgorithm": "lws",
  "rightInputs": [
    "696275"
  ],
  "sort": "ASC"
}

resp_direct = requests.post(serv + direct, json = json_direct, headers = token_dict)


In [157]:
resp_direct.json()

{u'content': [{u'concepts': [{u'id': u'2060394',
     u'name': u'XIST (homo sapiens)'},
    {u'id': u'696275', u'name': u'turner syndrome'}],
   u'relationships': [{u'concept0Id': u'2060394',
     u'concept0Name': u'XIST (homo sapiens)',
     u'concept1Id': u'696275',
     u'directionalTriples': [{u'object': u'696275',
       u'predicate': u'is associated with',
       u'subject': u'2060394',
       u'tripleId': u'129305211'},
      {u'object': u'696275',
       u'predicate': u'causes',
       u'subject': u'2060394',
       u'tripleId': u'129305187'}]}],
   u'score': 6.017065075043666}],
 u'first': True,
 u'last': True,
 u'number': 0,
 u'numberOfElements': 1,
 u'size': 10,
 u'sort': None,
 u'totalElements': 1,
 u'totalPages': 1}

In [151]:
json_direct_count = {
  "additionalFields": [
  ],
  "ids": [
    "2060394", "696275"
  ]
}

resp_direct_count = requests.post(serv + direct_count, json = json_direct_count, headers = token_dict)

In [153]:
print resp_direct_count.content

{}


## Problem 2

### Description
We need to find all meaningful connections between drugs and diseases and write them to the dictionary. We've chosen to retrieve drugs by Drugbank ID and "Drug" semantic category and diseases by it's name and semantic category "Disorder". 

In [5]:
def get_disease_ids(row):
    
    disease = row["Disease"]
    query = "term:%s" % disease
    json_disease = {"additionalFields": [], 
                 "queryString": query,
                 "semanticcategory": "Disorder",
                 "searchType": "TOKEN"}
    resp = rq.post(serv + search, json = json_disease, headers = token_dict)
    content = resp.json()["content"]
    disease_ids = [disease["id"] for disease in content]
    return disease_ids
    
def get_drug_ids(row):
    
    drugbankID = row["Drugbank ID"]
    query = "term:%s" % drugbankID
    json_drug = {"additionalFields": [], 
                 "queryString": query,
                 "semanticcategory": "Drug",
                 "searchType": "TOKEN",
                 "knowledgebase": "drugbank"}
    resp = rq.post(serv + search, json = json_drug, headers = token_dict)
    drug_ids = [drug["id"] for drug in resp.json()["content"]]
    return drug_ids

def get_relations(row):
    leftInputs = row["drug_ids"]
    rightInputs = row["disease_ids"]
    json_relations = {
                      "additionalFields": ["directionalTriples"],
                      "leftInputs": leftInputs,
                      "relationshipWeightAlgorithm": "PWS",
                      "rightInputs": rightInputs,
                      "sort": "ASC"
                    }
    rq.post(serv + direct, json = json_relations, headers = token_dict)
    return rq.json

In [8]:
drug_disease["drug_ids"] = drug_disease.apply(get_drug_ids, axis = 1)

In [6]:
drug_disease["disease_ids"] = drug_disease.apply(get_disease_ids, axis = 1)

In [9]:
drug_disease.head(20)

,Disease,Drug,Drugbank ID,Palliative,In FDA label,Relative Efficacy (RE),Indication (from DailyMed),Note,disease_ids,drug_ids
0,adenocarcinoma,Erlotinib,DB00530,0,1,"0,867925",TARCEVA is a kinase inhibitor indicated for: F...,"Nearly 40% of lung cancers are adenocarcinoma,...","[28278, 30784, 53678, 64924, 65672, 100160, 10...","[122609, 161896]"
1,adenocarcinoma,Floxuridine,DB00322,1,1,NaN,"Floxuridine for Injection, USP is effective in...",For palliative management of gastrointestinal ...,"[28278, 30784, 53678, 64924, 65672, 100160, 10...",[197550]
2,adenocarcinoma,Gefitinib,DB00317,0,1,"0,752137",IRESSA is a tyrosine kinase inhibitor indicate...,"Nearly 40% of lung cancers are adenocarcinoma,...","[28278, 30784, 53678, 64924, 65672, 100160, 10...",[5137]
3,alzheimer disease,Donepezil,DB00843,0,1,"0,538462",Donepezil hydrochloride is an acetylcholineste...,NaN,"[123503, 136417, 154026, 276854, 320236, 46891...","[528680, 1067868]"
4,alzheimer disease,Galantamine,DB00674,0,1,NaN,Galantamine hydrobromide tablet is a cholinest...,NaN,"[123503, 136417, 154026, 276854, 320236, 46891...",[282231]
5,alzheimer disease,Memantine,DB01043,0,1,"0,727273",Namenda (memantine hydrochloride) is indicated...,NaN,"[123503, 136417, 154026, 276854, 320236, 46891...","[586820, 1281846]"
6,alzheimer disease,Rivastigmine,DB00989,0,1,"0,860465",Rivastigmine tartrate capsules are an acetylch...,NaN,"[123503, 136417, 154026, 276854, 320236, 46891...","[284146, 694287]"
7,alzheimer disease,Selegiline,DB01037,0,0,NaN,Selegiline is indicated as an adjunct in the m...,NaN,"[123503, 136417, 154026, 276854, 320236, 46891...","[1150840, 2264168, 2665172]"
8,alzheimer disease,Vitamin E,DB00163,0,0,NaN,NaN,No corresponding FDA label,"[123503, 136417, 154026, 276854, 320236, 46891...","[630552, 851656, 1275452]"
9,amyotrophic lateral sclerosis,Riluzole,DB00740,0,1,"0,714286",Riluzole is indicated for the treatment of pat...,NaN,"[60997, 81651, 122285, 127011, 181615, 205724,...",[5537236]
